In [3]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier 

In [4]:
directory="D:\Purdue Courses\Fall Mod 1\MGMT 571 Data Mining using SAS Enterprise Miner\Project\\bankruptcy_Train"
directory_test="D:\Purdue Courses\Fall Mod 1\MGMT 571 Data Mining using SAS Enterprise Miner\Project\\bankruptcy_Test_X"

In [5]:
df=pd.read_csv(directory+"\\bankruptcy_Train.csv")


In [6]:
feature_cols=['Attr1','Attr2','Attr3','Attr4','Attr5','Attr6','Attr7','Attr8','Attr9','Attr10','Attr11','Attr12','Attr13','Attr14','Attr15','Attr16','Attr17','Attr18','Attr19','Attr20','Attr21','Attr22','Attr23','Attr24','Attr25','Attr26','Attr27','Attr28','Attr29','Attr30','Attr31','Attr32','Attr33','Attr34','Attr35','Attr36','Attr37','Attr38','Attr39','Attr40','Attr41','Attr42','Attr43','Attr44','Attr45','Attr46','Attr47','Attr48','Attr49','Attr50','Attr51','Attr52','Attr53','Attr54','Attr55','Attr56','Attr57','Attr58','Attr59','Attr60','Attr61','Attr62','Attr63','Attr64']

In [7]:
X = df[feature_cols] # Features
y = df['class'] # Target variable


In [8]:
df_test=pd.read_csv(directory_test+'\\bankruptcy_Test_X.csv')

In [9]:
test_final=df_test[feature_cols]

In [15]:
X_train1, X_test, y_train1, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)


In [38]:
from imblearn.under_sampling import NearMiss
nr = NearMiss()
X_train, y_train = nr.fit_sample(X_train, y_train)

In [39]:
np.bincount(y_train)

array([6850, 6850], dtype=int64)

In [29]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [30]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 57.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 111.5min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [31]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [32]:
rfc = RandomForestClassifier(n_estimators=400, max_depth=None, max_features='sqrt',min_samples_split=2, min_samples_leaf=1, bootstrap=False)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(test_final)
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.906384545538371


In [34]:
y_val_pred = rfc.predict_proba(X_test)
log_loss(y_test, y_val_pred)

0.09339488046558629

In [35]:
a=rfc.predict_proba(test_final)

In [36]:
np.savetxt(directory+"foo.csv", a, delimiter=",")

In [ ]:
rfc2 = RandomForestClassifier(n_estimators=2000, max_depth=30, criterion='gini', max_features='auto',min_samples_split=2, min_samples_leaf=4, bootstrap=False)
rfc2.fit(X_train,y_train)
rfc_predict = rfc.predict(test_final)
rfc_cv_score = cross_val_score(rfc2, X, y, cv=10, scoring='roc_auc')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

In [41]:
random_state = 1

In [42]:
rf1 = RandomForestClassifier(n_estimators=250, criterion='entropy',  n_jobs = -1,  random_state=random_state)
rf1.fit(X_train, y_train)
y_val_pred = rf1.predict_proba(X_test)
log_loss(y_test, y_val_pred)

0.12209420950535142

In [ ]:
gbc = GradientBoostingClassifier(random_state=random_state)
gbc.fit(X_train, y_train)
y_val_pred = gbc.predict_proba(X_test)
log_loss(y_test, y_val_pred)

In [ ]:
rf2 = RandomForestClassifier(n_estimators=250, criterion='gini',  n_jobs = -1,  random_state=random_state)
rf2.fit(X_train, y_train)
y_val_pred = rf1.predict_proba(X_test)
log_loss(y_test, y_val_pred)

In [ ]:
rfc_cv_score = cross_val_score(xgb, X, y, cv=10, scoring='roc_auc')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

In [ ]:
eclf = VotingClassifier(estimators=[
    ('rf1', rfc), ('rf2', rf2), ('gbc', gbc), ('xgb',xgb)], voting='soft', weights=[3,2,1,8])
eclf.fit(X_train, y_train)
y_val_pred = eclf.predict_proba(X_test)
log_loss(y_test, y_val_pred)

In [ ]:
a=eclf.predict_proba(test_final)

In [ ]:
np.savetxt(directory+"foo.csv", a, delimiter=",")

In [ ]:
rfc_cv_score = cross_val_score(eclf, X, y, cv=10, scoring='roc_auc')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())